Instalando SDK do Google

In [ ]:
!pip install -q -U google-generativeai

Configurar API_KEY

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('API_KEY')

Configurar genai object

In [ ]:
import google.generativeai as genai
genai.configure(api_key=API_KEY)

Configurações de segurança e modelo

In [ ]:
# configuracao padrao do gemini para modelo e seguranca
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

Instruções a AI a se comportar como um profissional de educação.

In [ ]:
m_instruction = "Você é um profissional experiente na educação de jovens e pre-adolescentes, especializado na BNCC (Base Nacional Comum Curricular) que descreve as habilidades e as aprendizagens essenciais que todos os alunos devem desenvolver durante cada etapa da educação básica.\n\nVocê receberá o resultado de uma prova, No formato Questão e Resposta.\nVocê deverá avaliar as respostas e relacioná-las com as competencias descritas na planilha.\n\nPor exemplo:\nQuestão: Ordene os seguintes numeros em ordem crescente (5,4,3,2,1).\nResposta: 2,5,4,1,3\n\nResultado: Fere a habilidade EF01MA01\n\nAo final, forneça uma breve sugestão de atividades que os pais possam realizar para auxiliar no desenvolvimento das competências que precisam de melhorias. \nTambém resuma brevemente os possíveis impactos futuros na vida prática da criança, causados pela falta desses conhecimentos.\nUtilize uma linguagem simples!"

Selecionar genai model

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

Criar funcao para leitura de planilha no googlesheets

In [ ]:
!pip install -q -U oauth2client

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

# Define the scope
scope = ['https://spreadsheets.google.com/feeds']

# Load credentials from the JSON file
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

# Build the Google Drive API service
drive = build('drive', 'v3', credentials=credentials)


In [ ]:
import pandas as pd
def read_gemini_spreadsheet():
  # read by default 1st sheet of an excel file
  dataframe1 = pd.read_excel('bncc.xlsx')

  print(dataframe1)

Preencher perguntas e respostas

In [ ]:
prompt_parts = []

# Iniciar um loop para que o usuario possa informar as perguntas e respostas
print("Por favor informe os seguintes dados: ")
name = input("Nome do(a) aluno(a): ")
age = input("Idade do(a) aluno(a): ")
grade = input("Serie do(a) aluno(a): ")
print("Agora vamos começar informando as perguntas de respostas.")
print('digite "fim" para encerrar e iniciar a analise.');
prompt = ''
prompt_input = '';
answer = ''
question = ''

while prompt != 'fim':

  if not question:
    prompt = input('Digite a pergunta ou "fim" para sair: ')
  else:
    prompt = input('Digite a resposta ou "fim" para sair: ')


  if not question:
    question = f"Pergunta {len(prompt_parts) + 1}: {prompt}"
  else:
    if prompt != 'fim':
      question = f"{question} \nResposta: {prompt}"
      prompt_parts.append(question)
      question = ''
      answer = ''

# Exibir lista com perguntas enviadas
print(f"Você informou {len(prompt_parts)} pergunta(s):\n")

# Print each item in the list
for item in prompt_parts:
  print(item)

prompt_parts.append(f"Nome: {name} \n")
prompt_parts.append(f"Idade: {age} \n")
prompt_parts.append(f"Serio: {grade} do ensino fundamental\n")


Por favor informe os seguintes dados: 
Nome do(a) aluno(a): quanto e 1 + 1
Idade do(a) aluno(a): 12
Serie do(a) aluno(a): 2
Agora vamos começar informando as perguntas de respostas.
digite "fim" para encerrar e iniciar a analise.
Digite a pergunta ou "fim" para sair: quanto e 1+1
Digite a resposta ou "fim" para sair: 3
Digite a pergunta ou "fim" para sair: fim
Você informou 1 pergunta(s):

Pergunta 1: quanto e 1+1 
Resposta: 3


In [ ]:
prompt_parts.append(read_gemini_spreadsheet())

Exibir relatório

In [ ]:
response = model.generate_content(prompt_parts)
print(response.text)

## Análise da resposta do aluno:

**Questão:** Quanto é 1 + 1?
**Resposta:** 3

**Habilidade:** EF01MA06 (Resolver e elaborar problemas de adição e subtração, envolvendo números de até 10, utilizando objetos, desenhos, números e sinais, oralmente e por escrito, em situações diversas, com ou sem apoio de material manipulável.)

**Resultado:** A resposta está incorreta. O aluno demonstra dificuldade com a operação básica de adição,  não compreendendo o conceito de somar quantidades.

## Sugestões de atividades para os pais:

* **Brincar com jogos de tabuleiro:**  Estimulem o desenvolvimento do raciocínio matemático com jogos simples como dominó, dados e jogos de trilha, que envolvem contagem e adição.
* **Usar objetos concretos para contar:**  Façam atividades práticas usando brinquedos, frutas ou outros objetos do dia a dia para ensinar a criança a contar e somar quantidades.
* **Criar histórias matemáticas:** Invente histórias simples que envolvam situações de adição, como "João tinha 